# British National Grid Helper Functions
---

*Intended to support use cases related to statisitical binning against 1 kilometre tile references and use of 1 kilometre tile references as a pseudo-grid index.*

### Dependencies
---

* [Shapely](https://shapely.readthedocs.io/en/latest/manual.html) - A hard dependency of GeoPandas. From the documentation: a Python package for set-theoretic analysis and manipulation of planar features using (via Python’s ctypes module) functions from the well known and widely deployed GEOS library.

In [3]:
import math
import numpy as np
import pandas as pd
from itertools import product
from shapely import errors, wkt as _wkt_ # Alias used to distinguish between WKT input
from shapely.validation import explain_validity

### 01 - togridref()
---

Python function to return a 1 metre British National Grid grid reference from an input easting and northing coordinate pair.

In [5]:
def togridref(e, n):
    '''
    Return a 1 metre British National Grid grid reference from an input easting and northing coordinate pair.
    
    Assumes projected bounds ((0,0), (700000, 1300000).
    
    Only easting coordinates in the range 0 <= e < 700000 are supported.
    
    Only northing coordinates in the range 0 <= e < 1300000 are supported.
    
    Python implementation of Tim Manners' Javascript helper function
    https://github.com/OrdnanceSurvey/os-transform/blob/f133372dcee7a34c85dd51999c90171a884b3bc0/os-transform.js#L41

    Args:
        e (int/float): Easting cordinate.
        n (int/float): Northing cordinate.

    Returns:
        gridref (string): The return grid reference value. 
    '''    
    # BNG 100 kilometre prefixes
    prefixes = np.array([['SV','SW','SX','SY','SZ','TV','TW'],
                         ['SQ','SR','SS','ST','SU','TQ','TR'],
                         ['SL','SM','SN','SO','SP','TL','TM'],
                         ['SF','SG','SH','SJ','SK','TF','TG'],
                         ['SA','SB','SC','SD','SE','TA','TB'],
                         ['NV','NW','NX','NY','NZ','OV','OW'],
                         ['NQ','NR','NS','NT','NU','OQ','OR'],
                         ['NL','NM','NN','NO','NP','OL','OM'],
                         ['NF','NG','NH','NJ','NK','OF','OG'],
                         ['NA','NB','NC','ND','NE','OA','OB'],
                         ['HV','HW','HX','HY','HZ','JV','JW'],
                         ['HQ','HR','HS','HT','HU','JQ','JR'],
                         ['HL','HM','HN','HO','HP','JL','JM']])
    
    # Transform input coordinates to support return of prefix
    x = math.floor(e / 100000)
    y = math.floor(n / 100000)
    
    # Negative easting or northing coordinates are outside of the supported BNG bounds
    if x >= 0 and y >= 0:
        try:
            prefix = prefixes[y][x]
        # Catch exception as indicator that input coordinates are out of bounds
        except IndexError:
            # Return out of bounds notice to user
            gridref = '=> Out of Bounds - Coordinates {}, {} Outside of BNG Bounds'.format(str(e), str(n))
        else:
            x = math.floor(e % 100000)
            y = math.floor(n % 100000)
            
            x = str(x).zfill(5)
            y = str(y).zfill(5)
            
            # Format grid reference
            gridref = prefix + ' ' + x + ' ' + y
    else:
        # Return out of bounds notice to user
        gridref = '=> Out of Bounds - Coordinates {}, {} Outside of BNG Bounds'.format(str(e), str(n))

    return gridref

### 02 - totileref()
---

Python function to return a 1 kilometre British National Grid tile reference from an input easting and northing coordinate pair.

In [7]:
def totileref(e, n):
    '''
    Return a 1 kilometre British National Grid tile reference from an input easting and northing coordinate pair.

    Args:
        e (int/float): Easting cordinate.
        n (int/float): Northing cordinate.

    Returns:
        tileref (string): The return grid reference value. 
    '''  
    # Return 1m BNG grid reference
    gridref = togridref(e, n)
    
    # Test length of grid reference to flag out of bounds input coordinates
    if len(gridref) == 14:
        # Format tile reference
        li = togridref(e, n).split()
        
        tileref = ''.join(i[0:2] for i in li)
    else:
        # Return out of bounds notice to user
        tileref = gridref
    
    return tileref

### 03 - toboundstileref()
---

Python function to return the 1 kilometre British National Grid tile references for the axis-orientated bounding box of a geometry:
<br>
<br>
* Attempt to parse well known text (WKT) representation of geometry.
* Validate WKT input - where invalid report reason.
* Validate geometry using GEOS implementation - where invalid report reason.
* Report out-of-bounds BNG occurances whilst returning all in-bounds tiles for a geometry.
* Errors reported using `=> ` prefix at start of return value. One of `=> Out of Bounds - `, `=> Invalid WKT - ` and `=> Invalid geometry - `.

In [9]:
def _ceil_(x):
    '''
    Custom ceiling function for use in boundstileref function.

    Args:
        x (float): Numeric value.

    Returns:
        (int) 
    '''  
    return math.ceil(x / 1000) * 1000
  
def _floor_(x):
    '''
    Custom ceiling function for use in boundstileref function.

    Args:
        x (float): Numeric value.

    Returns:
        (int) 
    '''  
    return math.floor(x / 1000) * 1000
  
def toboundstileref(wkt):
    '''
    Return the 1 kilometre British National Grid tile references for the axis-orientated bounding box of a geometry.

    Args:
        wkt (string): Well known text representation of geometry.

    Returns:
        tileref (list(string)): The return tile reference string values in a list.
    '''  
    # Initialise tileref list
    tileref = []
    
    # Attempt Shapely geometry creation from WKT representation
    try:
        s = _wkt_.loads(wkt)
    # Catch WKTReadingError exception validating WKT input
    except errors.WKTReadingError as err:
        # Return exception to user
        tileref.append('=> Invalid WKT - ' + str(err))
    else:    
        # Test geometry validity
        if s.is_valid == False:
            # Return invalid explanation notice to user
            tileref.append('=> Invalid geometry - ' + explain_validity(s))
        # Point geometry presents simple case
        elif s.type == 'Point':
            x, y = s.xy
            tileref.append(totileref(x[0], y[0]))
        # All other geometry types require BBOX derviation
        else:
            # Compute BBOX bounds
            bounds = list(s.bounds)
            # Lower left and upper right corners rounded down and up respectively
            bounds[0], bounds[1] = _floor_(bounds[0]), _floor_(bounds[1])
            bounds[2], bounds[3] = _ceil_(bounds[2]), _ceil_(bounds[3]) 
            # Evaluate product of bounds to dervie all 1 kilometre tile references
            x1, y1, x2, y2 = bounds
            for x, y in list(product(range(x1, x2, 1000), range(y1, y2, 1000))):
              tileref.append(totileref(x, y))
    
    return tileref